<h1>삼성전자(005930) 재무제표를 MySQL DB에 넣기</h1>

In [1]:
import requests
from io import BytesIO
import pandas as pd
import numpy as np
import re
import os
import mysql.connector
from sqlalchemy import create_engine
from datetime import datetime

In [34]:
# opendart API로 상장사 사업보고서의 rcp_no 불러오기 
api_key = '27d86452b033fe973df6f1c980c37a2586801a1a'
corp_code = '00126380' # 삼성전자 corp_code
bgn_de = datetime.today().replace(year=2017).strftime('%Y%m%d')
end_de = datetime.today().strftime('%Y%m%d')

url = 'https://opendart.fss.or.kr/api/list.xml?crtfc_key={}&corp_code={}&bgn_de={}&end_de={}&last_reprt_at=Y&pblntf_detail_ty=A001&page_no=1&page_count=10'.format(api_key, corp_code, bgn_de, end_de)

In [35]:
resp = requests.get(url)
webpage = resp.content.decode('UTF-8')
webpage

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><result><status>000</status><message>정상</message><page_no>1</page_no><page_count>10</page_count><total_count>3</total_count><total_page>1</total_page><list><corp_code>00126380</corp_code><corp_name>삼성전자</corp_name><stock_code>005930</stock_code><corp_cls>Y</corp_cls><report_nm>사업보고서 (2018.12)</report_nm><rcept_no>20190401004781</rcept_no><flr_nm>삼성전자</flr_nm><rcept_dt>20190401</rcept_dt><rm>연</rm></list><list><corp_code>00126380</corp_code><corp_name>삼성전자</corp_name><stock_code>005930</stock_code><corp_cls>Y</corp_cls><report_nm>사업보고서 (2017.12)</report_nm><rcept_no>20180402005019</rcept_no><flr_nm>삼성전자</flr_nm><rcept_dt>20180402</rcept_dt><rm>연</rm></list><list><corp_code>00126380</corp_code><corp_name>삼성전자</corp_name><stock_code>005930</stock_code><corp_cls>Y</corp_cls><report_nm>사업보고서 (2016.12)</report_nm><rcept_no>20170331004518</rcept_no><flr_nm>삼성전자</flr_nm><rcept_dt>20170331</rcept_dt><rm>연</rm></list></result>'

In [36]:
stock_code = re.findall(r'<stock_code>(.*?)</stock_code>', webpage)[0]
rcp_no_list = re.findall(r'<rcept_no>(.*?)</rcept_no>', webpage)
period_list = re.findall(r'<report_nm>사업보고서 \((.*?)\)</report_nm>', webpage)
stock_code

'005930'

In [37]:
# 각 rcp_no의 pair number인 dcm_no 조회
dcm_no_list = []
for rcp_no in rcp_no_list:
    resp = requests.get('http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp_no))
    webpage = resp.content.decode('utf-8')
    dcm_no = re.findall(r"'{}', '(.*?)',".format(rcp_no), webpage)[0]
    dcm_no_list.append(dcm_no)
    
dcm_no_list

['6616741', '6060273', '5540271']

In [38]:
# 파악된 rcp_no와 dcm_no 정보를 url에 넣어 재무제표 excel 파일을 pandas로 불러오기
user_agent = "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"

url = "http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={}&dcm_no={}&lang=ko".format(rcp_no_list[0],dcm_no_list[0])
resp = requests.get(url, headers={"user-agent": user_agent})
table = BytesIO(resp.content)

# 재무제표별 DataFrame 생성
df_bs_con = pd.read_excel(table, sheet_name="연결 재무상태표", skiprows=6)
df_bs_sep = pd.read_excel(table, sheet_name="재무상태표", skiprows=6)
df_is_con = pd.read_excel(table, sheet_name="연결 손익계산서", skiprows=6)
df_is_sep = pd.read_excel(table, sheet_name="손익계산서", skiprows=6)
df_cf_con = pd.read_excel(table, sheet_name="연결 현금흐름표", skiprows=6)
df_cf_sep = pd.read_excel(table, sheet_name="현금흐름표", skiprows=6)
df_ci_con = pd.read_excel(table, sheet_name="연결 포괄손익계산서", skiprows=6)
df_ci_sep = pd.read_excel(table, sheet_name="포괄손익계산서", skiprows=6)


In [39]:
# 계정 컬럼명을 account로 명명
period_list.insert(0, 'account')
period_list

['account', '2018.12', '2017.12', '2016.12']

In [42]:
# 각 df별 column 설정
df_bs_con.columns = period_list
df_bs_sep.columns = period_list
df_is_con.columns = period_list
df_is_sep.columns = period_list 
df_cf_con.columns = period_list
df_cf_sep.columns = period_list
df_ci_con.columns = period_list
df_ci_sep.columns = period_list

In [43]:
# 계정명 내 공백제거
df_bs_con['account'] = df_bs_con['account'].str.strip()
df_bs_sep['account'] = df_bs_sep['account'].str.strip()
df_is_con['account'] = df_is_con['account'].str.strip()
df_is_sep['account'] = df_is_sep['account'].str.strip()
df_cf_con['account'] = df_cf_con['account'].str.strip()
df_cf_sep['account'] = df_cf_sep['account'].str.strip()
df_ci_con['account'] = df_ci_con['account'].str.strip()
df_ci_sep['account'] = df_ci_sep['account'].str.strip()

In [44]:
# 공백란을 0로 채우기
df_bs_con = df_bs_con.replace(" ", 0)
df_bs_sep = df_bs_sep.replace(" ", 0)
df_is_con = df_is_con.replace(" ", 0)
df_is_sep = df_is_sep.replace(" ", 0)
df_cf_con = df_cf_con.replace(" ", 0)
df_cf_sep = df_cf_sep.replace(" ", 0)
df_ci_con = df_cf_con.replace(" ", 0)
df_ci_sep = df_cf_sep.replace(" ", 0)

In [15]:
# 컬럼별 type 확인
df_cf_sep.dtypes

account    object
2018.12     int64
2017.12     int64
2016.12     int64
dtype: object

In [45]:
#connect MySQL
mydb = mysql.connector.connect(host = 'localhost',
                                user = 'root',
                                passwd = 'root',
                                db = 'fsdb')
mycursor = mydb.cursor()

In [17]:
# BS tables 만들기(consolidated, seperated)
def create_bs_tables(stock_code):
    table_list = [df_bs_con, df_bs_sep]
    table_tuple = ('bs_con', 'bs_sep')

    for table, table_name in zip(table_list, table_tuple):
        # DB에 테이블 만들기
        sql = """CREATE TABLE {}_{} (
                        id bigint(20) unsigned NOT NULL AUTO_INCREMENT,
                        account varchar(255),
                        FY18 int(20),
                        FY17 int(20),
                        FY16 int(20),
                        PRIMARY KEY (id));""".format(stock_code, table_name)
        
        mycursor.execute(sql)

        # 각 테이블에 관련 계정명과 수치 입력
        for i in table.index:
            sql2 = """INSERT INTO {}_{} (account, FY18, FY17, FY16) 
                    VALUES ("{}", {}, {}, {})""".format(stock_code, table_name, table.loc[i,table.columns[0]], table.loc[i,table.columns[1]], table.loc[i,table.columns[2]], table.loc[i,table.columns[3]])
            mycursor.execute(sql2)
        mydb.commit()
    
    return "{} BS tables 생성 성공".format(stock_code)

In [18]:
# bs table 생성
create_bs_tables(stock_code)

'005930 BS table 생성 성공'

In [19]:
# bs table 확인
sql = "SELECT * FROM {}_{}".format(stock_code, 'bs_sep')
pd.read_sql_query(sql, mydb)

,id,account,FY18,FY17,FY16
0,1,자산,0,0,0
1,2,유동자산,80039455,70155189,69981128
2,3,현금및현금성자산,2607957,2763768,3778371
3,4,단기금융상품,34113871,25510064,30170656
4,5,매출채권,24933267,27881777,23514012
5,6,미수금,1515079,2201402,2319782
6,7,선급금,807262,1097598,814300
7,8,선급비용,2230628,2281179,2375520
8,9,재고자산,12440951,7837144,5981634
9,10,기타유동자산,1390440,582257,743163


In [20]:
# IS tables 만들기(consolidated, seperated)
def create_is_tables(stock_code):
    table_list = [df_is_con, df_is_sep]
    table_tuple = ('is_con', 'is_sep')
    
    for table, table_name in zip(table_list, table_tuple):
        # DB에 테이블 만들기
        sql = """CREATE TABLE {}_{} (
                            id bigint(20) unsigned NOT NULL AUTO_INCREMENT,
                            account varchar(255),
                            FY18 int(20),
                            FY17 int(20),
                            FY16 int(20),
                            PRIMARY KEY (id));""".format(stock_code, table_name)
        mycursor.execute(sql)
    
    # 각 테이블에 관련 계정명과 수치 입력
    for i in table.index:
        sql2 = """INSERT INTO {}_{} (account, FY18, FY17, FY16) 
                VALUES ("{}", {}, {}, {})""".format(stock_code, table_name, table.loc[i,table.columns[0]], table.loc[i,table.columns[1]], table.loc[i,table.columns[2]], table.loc[i,table.columns[3]])
        mycursor.execute(sql2)
    mydb.commit()
    return "{} IS tables 생성 성공".format(stock_code)

In [21]:
create_is_tables(stock_code)

'005930_is_con 생성 성공'

In [32]:
df_ci_con

,account,2018.12,2017.12,2016.12
0,당기순이익(손실),44344857,42186747,22726092
1,기타포괄손익,-12242,-5502257,1991400
2,후속적으로 당기손익으로 재분류되지 않는 포괄손익,-656647,407900,1014040
3,기타포괄손익-공정가치금융자산평가손익,-235865,,
4,관계기업 및 공동기업의 기타포괄손익에 대한 지분,-10631,-6347,50438
5,순확정급여부채 재측정요소,-410151,414247,963602
6,후속적으로 당기손익으로 재분류되는 포괄손익,644405,-5910157,977360
7,매도가능금융자산평가손익,,511207,-23839
8,관계기업 및 공동기업의 기타포괄손익에 대한 지분,6688,-49256,-130337
9,해외사업장환산외환차이,590638,-6334987,1131536


In [22]:
# is table 확인
sql = "SELECT * FROM {}_{}".format(stock_code, 'is_sep')
pd.read_sql_query(sql, mydb)

,id,account,FY18,FY17,FY16
0,1,수익(매출액),170381870,161915007,133947204
1,2,매출원가,101666506,101399657,97290644
2,3,매출총이익,68715364,60515350,36656560
3,4,판매비와관리비,25015913,25658259,23009124
4,5,영업이익(손실),43699451,34857091,13647436
5,6,기타수익,972145,2767967,2185600
6,7,기타비용,504562,1065014,1289594
7,8,금융수익,3737494,4075602,5803751
8,9,금융비용,3505673,4102094,5622119
9,10,법인세비용차감전순이익(손실),44398855,36533552,14725074


In [23]:
# CF table 만들기(consolidated, seperated)
def create_cf_tables(stock_code):
    table_list = [df_cf_con, df_cf_sep]
    table_tuple = ('cf_con', 'cf_sep')
    
    for table, table_name in zip(table_list, table_tuple):
        # DB에 자산, 부채, 자본항목별 테이블 생성
        sql = """CREATE TABLE {}_{} (
                        id bigint(20) unsigned NOT NULL AUTO_INCREMENT,
                        account varchar(255),
                        FY18 int(20),
                        FY17 int(20),
                        FY16 int(20),
                        PRIMARY KEY (id));""".format(stock_code, table_name)

        mycursor.execute(sql)

        # 각 행별 정보 입력
        for i in table.index:
            sql2 = """INSERT INTO {}_{} (account, FY18, FY17, FY16) 
                    VALUES ("{}", {}, {}, {})""".format(stock_code, table_name, table.loc[i,table.columns[0]], table.loc[i,table.columns[1]], table.loc[i,table.columns[2]], table.loc[i,table.columns[3]])
            mycursor.execute(sql2)
        mydb.commit()
    
    return "{} CF tables 생성 성공".format(stock_code)

In [24]:
# cf tables 생성
create_cf_tables(stock_code)

'005930 CF tables 생성 성공'

In [25]:
# cf table 확인
sql = "SELECT * FROM {}_{}".format(stock_code, 'cf_sep')
pd.read_sql_query(sql, mydb)

,id,account,FY18,FY17,FY16
0,1,영업활동 현금흐름,44341217,38906190,23984804
1,2,영업에서 창출된 현금흐름,53596311,41350471,24901464
2,3,당기순이익,32815127,28800837,11579749
3,4,조정,27095149,18012976,14910093
4,5,영업활동으로 인한 자산부채의 변동,-6313965,-5463342,-1588378
5,6,이자의 수취,459074,491501,622118
6,7,이자의 지급,-343270,-265364,-208010
7,8,배당금 수입,779567,1118779,903758
8,9,법인세 납부액,-10150465,-3789197,-2234526
9,10,투자활동 현금흐름,-31678548,-28118806,-14240450


In [46]:
# comprehesive income table 만들기(consolidated, seperated)
def create_ci_tables(stock_code):
    table_list = [df_ci_con, df_ci_sep]
    table_tuple = ('ci_con', 'ci_sep')
    
    for table, table_name in zip(table_list, table_tuple):
        # DB에 ci 테이블 생성
        sql = """CREATE TABLE {}_{} (
                        id bigint(20) unsigned NOT NULL AUTO_INCREMENT,
                        account varchar(255),
                        FY18 int(20),
                        FY17 int(20),
                        FY16 int(20),
                        PRIMARY KEY (id));""".format(stock_code, table_name)

        mycursor.execute(sql)

        # 각 행별 정보 입력
        for i in table.index:
            sql2 = """INSERT INTO {}_{} (account, FY18, FY17, FY16) 
                    VALUES ("{}", {}, {}, {})""".format(stock_code, table_name, table.loc[i,table.columns[0]], table.loc[i,table.columns[1]], table.loc[i,table.columns[2]], table.loc[i,table.columns[3]])
            mycursor.execute(sql2)
        mydb.commit()
    
    return "{} CI tables 생성 성공".format(stock_code)

In [48]:
# ci tables 생성
create_ci_tables(stock_code)

'005930 CI tables 생성 성공'

In [49]:
# ci table 확인
sql = "SELECT * FROM {}_{}".format(stock_code, 'ci_sep')
pd.read_sql_query(sql, mydb)

,id,account,FY18,FY17,FY16
0,1,영업활동 현금흐름,44341217,38906190,23984804
1,2,영업에서 창출된 현금흐름,53596311,41350471,24901464
2,3,당기순이익,32815127,28800837,11579749
3,4,조정,27095149,18012976,14910093
4,5,영업활동으로 인한 자산부채의 변동,-6313965,-5463342,-1588378
5,6,이자의 수취,459074,491501,622118
6,7,이자의 지급,-343270,-265364,-208010
7,8,배당금 수입,779567,1118779,903758
8,9,법인세 납부액,-10150465,-3789197,-2234526
9,10,투자활동 현금흐름,-31678548,-28118806,-14240450


In [50]:
mydb.close()